In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import sqlite3
import random
import os

random.seed(137)
np.random.seed(137)

pd.options.mode.chained_assignment = None  # default="warn"
%matplotlib inline

## Read data

In [2]:
raw_data_dir = "data/raw/mimic-iv/mimic-iv-2.2"
processed_data_dir = "data/processed"

### read icu stays

In [3]:
icustays = pd.read_csv(os.path.join(raw_data_dir, "icu/icustays.csv"))
icustays["intime"] = pd.to_datetime(icustays["intime"], errors="coerce")
icustays["outtime"] = pd.to_datetime(icustays["outtime"], errors="coerce")
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
73177,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
73178,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
73179,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [4]:
icustays = icustays[["subject_id", "hadm_id", "stay_id", "intime", "outtime", "los"]]
icustays

,subject_id,hadm_id,stay_id,intime,outtime,los
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...
73176,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
73177,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
73178,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
73179,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [5]:
len(icustays["subject_id"].unique())

50920

### read admission for survival status

In [6]:
admissions = pd.read_csv(os.path.join(raw_data_dir, "hosp/admissions.csv"))
admissions["admittime"] = pd.to_datetime(admissions["admittime"], errors="coerce")
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"], errors="coerce")
admissions["deathtime"] = pd.to_datetime(admissions["deathtime"], errors="coerce")
admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [7]:
admissions = admissions[["subject_id", "hadm_id", "deathtime", "race", "hospital_expire_flag"]]
admissions

,subject_id,hadm_id,deathtime,race,hospital_expire_flag
0,10000032,22595853,NaT,WHITE,0
1,10000032,22841357,NaT,WHITE,0
2,10000032,25742920,NaT,WHITE,0
3,10000032,29079034,NaT,WHITE,0
4,10000068,25022803,NaT,WHITE,0
...,...,...,...,...,...
431226,19999828,25744818,NaT,WHITE,0
431227,19999828,29734428,NaT,WHITE,0
431228,19999840,21033226,2164-09-17 13:42:00,WHITE,1
431229,19999840,26071774,NaT,WHITE,0


### gender

In [8]:
# all patients are older than 18
patients = pd.read_csv(os.path.join(data_dir, "hosp/patients.csv"))
patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,NaN
299708,19999829,F,28,2186,2008 - 2010,NaN
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,NaN


In [9]:
len(patients["subject_id"].unique())

299712

In [10]:
patients["gender"].value_counts()

F    158553
M    141159
Name: gender, dtype: int64

In [11]:
patients.loc[patients["gender"]=="M", "gender"] = 1
patients.loc[patients["gender"]=="F", "gender"] = 0

In [12]:
icustays = icustays.merge(patients[["subject_id", "gender"]], on="subject_id", how="left")
icustays

,subject_id,hadm_id,stay_id,intime,outtime,los,gender
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,0
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,0
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,0
...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1
73177,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1
73178,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0
73179,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1


### age

In [13]:
age = pd.read_csv(os.path.join(demographics_dir, "age.csv"))
age

,subject_id,hadm_id,admittime,anchor_age,anchor_year,age
0,14962874,22614141,2165-10-15 15:59:00,89,2165,89
1,13241600,22574379,2162-10-29 15:36:00,89,2162,89
2,12945423,20185299,2122-07-08 05:40:00,89,2122,89
3,16014534,21633959,2176-03-03 14:33:00,89,2175,90
4,16014534,27188276,2176-03-27 11:53:00,89,2175,90
...,...,...,...,...,...,...
431226,12545126,27135177,2187-11-01 15:47:00,91,2187,91
431227,17579295,26848807,2162-11-16 07:15:00,91,2162,91
431228,18563244,26485584,2136-08-18 14:48:00,91,2136,91
431229,14865704,29199399,2156-11-12 13:59:00,91,2156,91


In [14]:
len(age["subject_id"].unique()), len(age["hadm_id"].unique())

(180733, 431231)

In [15]:
icustays = icustays.merge(age[["subject_id", "hadm_id", "age"]], on=["subject_id", "hadm_id"], how="left")
icustays

,subject_id,hadm_id,stay_id,intime,outtime,los,gender,age
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,0,55
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,0,55
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,0,46
...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1,43
73177,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1,82
73178,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48
73179,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1,58


### comorbidity

In [16]:
comorbidity = pd.read_csv(os.path.join(comorbidity_dir, "mimic_iv_comorbidity_admissions.csv"))
comorbidity = comorbidity.drop(columns=["index"], axis=1)
comorbidity

,hadm_id,mi,chf,pvd,cevd,dementia,cpd,rheumd,pud,mld,diab,diabwc,hp,rend,canc,msld,metacanc,aids,CCI
0,20000019,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,2
1,20000024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20000034,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,6
3,20000041,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,20000057,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430847,29999723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
430848,29999745,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
430849,29999809,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,2
430850,29999828,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [17]:
icustays = icustays.merge(comorbidity, on=["hadm_id"], how="left")
icustays

,subject_id,hadm_id,stay_id,intime,outtime,los,gender,age,mi,chf,...,mld,diab,diabwc,hp,rend,canc,msld,metacanc,aids,CCI
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,0,46,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1,43,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0
73177,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1,82,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
73178,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
73179,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1,58,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
icustays["CCI"].isna().sum()

22

### height

In [19]:
height = pd.read_csv(os.path.join(demographics_dir, "height.csv"))
height

,subject_id,stay_id,charttime,height
0,18918742,37516352,2172-11-16 09:52:00,177.0
1,18684109,39802825,2179-10-10 20:05:00,153.0
2,16356099,32742724,2123-10-16 14:37:00,171.0
3,17601497,39226753,2118-07-28 09:53:00,184.0
4,11071390,36457462,2114-04-25 08:51:00,196.0
...,...,...,...,...
33469,11463939,31023514,2172-07-07 14:04:00,147.0
33470,16326868,37604901,2170-04-27 10:47:00,147.0
33471,17334551,39238232,2136-08-31 18:21:00,147.0
33472,16303373,34089704,2120-06-18 09:24:00,147.0


In [20]:
height["height"].describe()

count    33474.000000
mean       169.452966
std         10.754778
min        122.000000
25%        163.000000
50%        170.000000
75%        178.000000
max        226.000000
Name: height, dtype: float64

In [21]:
height["height"].isna().sum()

0

In [22]:
len(height["subject_id"].unique()), len(height["stay_id"].unique())

(28478, 33474)

In [23]:
icustays = icustays.merge(height[["subject_id", "stay_id", "height"]], on=["subject_id", "stay_id"], how="left")
icustays

,subject_id,hadm_id,stay_id,intime,outtime,los,gender,age,mi,chf,...,diab,diabwc,hp,rend,canc,msld,metacanc,aids,CCI,height
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,NaN
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,0,46,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,157.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1,43,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,193.0
73177,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1,82,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,NaN
73178,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN
73179,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1,58,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,178.0


### weight duration

In [24]:
# weight is in kg
weight_duration = pd.read_csv(os.path.join(demographics_dir, "weight_duration.csv"))
weight_duration["starttime"] = pd.to_datetime(weight_duration["starttime"], errors="coerce")
weight_duration["endtime"] = pd.to_datetime(weight_duration["endtime"], errors="coerce")
weight_duration

,stay_id,starttime,endtime,weight,weight_type
0,30006067,2161-04-14 05:44:00,2161-04-15 22:10:58,75.5,admit
1,30006833,2156-07-30 16:50:53,2156-07-31 20:49:34,83.0,admit
2,30011071,2177-07-07 18:37:00,2177-07-07 20:00:00,113.0,admit
3,30011071,2177-07-07 20:00:00,2177-07-09 06:00:00,113.0,daily
4,30011071,2177-07-09 06:00:00,2177-07-11 02:03:47,110.6,daily
...,...,...,...,...,...
272442,37258075,2192-09-18 06:10:52,2192-09-19 06:00:00,79.8,daily
272443,34388423,2176-06-20 10:22:38,2176-06-28 06:00:00,86.6,daily
272444,31841687,2191-08-01 17:57:15,2191-08-02 04:00:00,65.2,daily
272445,30155363,2164-04-08 09:43:11,2164-04-08 19:00:00,52.8,daily


In [25]:
len(weight_duration["stay_id"].unique())

70689

### sofa scores

In [26]:
sofa_scores = pd.read_csv("/N/project/waveform_mortality/shared/datasets/Sofa_MIMIC_EICU/mimic_iv_hoursly_sofa_scores.csv")
sofa_scores

,stay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,30004144,0,2126-04-04 13:00:00,2126-04-04 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0,0,0,0,1,0,1
1,30004144,1,2126-04-04 14:00:00,2126-04-04 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,NaN,0,0,0,0,1,0,1
2,30004144,2,2126-04-04 15:00:00,2126-04-04 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,NaN,0,0,0,0,1,0,1
3,30004144,3,2126-04-04 16:00:00,2126-04-04 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,NaN,0,0,0,0,1,0,1
4,30004144,4,2126-04-04 17:00:00,2126-04-04 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,NaN,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043897,39993298,35,2189-08-03 11:00:00,2189-08-03 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0,0,0,0,0,0,0
6043898,39993298,36,2189-08-03 12:00:00,2189-08-03 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
6043899,39993298,37,2189-08-03 13:00:00,2189-08-03 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
6043900,39993298,38,2189-08-03 14:00:00,2189-08-03 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,0,0,0,0,0,0


In [27]:
sofa_scores.columns

Index(['stay_id', 'hr', 'starttime', 'endtime', 'pao2fio2ratio_novent',
       'pao2fio2ratio_vent', 'rate_epinephrine', 'rate_norepinephrine',
       'rate_dopamine', 'rate_dobutamine', 'meanbp_min', 'gcs_min', 'uo_24hr',
       'bilirubin_max', 'creatinine_max', 'platelet_min', 'respiration',
       'coagulation', 'liver', 'cardiovascular', 'cns', 'renal',
       'respiration_24hours', 'coagulation_24hours', 'liver_24hours',
       'cardiovascular_24hours', 'cns_24hours', 'renal_24hours',
       'sofa_24hours'],
      dtype='object')

In [28]:
len(sofa_scores["stay_id"].unique())

73163

In [29]:
sofa_scores = sofa_scores[["stay_id", "starttime", "endtime", "respiration", "coagulation", "liver", "cardiovascular", "cns", "renal", "sofa_24hours"]]
sofa_scores["starttime"] = pd.to_datetime(sofa_scores["starttime"], errors="coerce")
sofa_scores["endtime"] = pd.to_datetime(sofa_scores["endtime"], errors="coerce")
sofa_scores

,stay_id,starttime,endtime,respiration,coagulation,liver,cardiovascular,cns,renal,sofa_24hours
0,30004144,2126-04-04 13:00:00,2126-04-04 14:00:00,NaN,0.0,NaN,0.0,1.0,0.0,1
1,30004144,2126-04-04 14:00:00,2126-04-04 15:00:00,NaN,NaN,NaN,0.0,1.0,NaN,1
2,30004144,2126-04-04 15:00:00,2126-04-04 16:00:00,NaN,NaN,NaN,0.0,1.0,NaN,1
3,30004144,2126-04-04 16:00:00,2126-04-04 17:00:00,NaN,NaN,NaN,0.0,1.0,NaN,1
4,30004144,2126-04-04 17:00:00,2126-04-04 18:00:00,NaN,NaN,NaN,0.0,1.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...
6043897,39993298,2189-08-03 11:00:00,2189-08-03 12:00:00,NaN,NaN,NaN,NaN,0.0,NaN,0
6043898,39993298,2189-08-03 12:00:00,2189-08-03 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0
6043899,39993298,2189-08-03 13:00:00,2189-08-03 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0
6043900,39993298,2189-08-03 14:00:00,2189-08-03 15:00:00,NaN,NaN,NaN,0.0,NaN,NaN,0


In [29]:
# check sofa scores 24 hours
# sofa_scores[sofa_scores["stay_id"]==39993298]

In [30]:
sofa_scores.isnull().sum()

stay_id                 0
starttime               0
endtime                 0
respiration       5809939
coagulation       5666765
liver             5935156
cardiovascular     690538
cns               4436834
renal             3443509
sofa_24hours            0
dtype: int64

In [31]:
sofa_scores = sofa_scores.fillna(0)
sofa_scores

,stay_id,starttime,endtime,respiration,coagulation,liver,cardiovascular,cns,renal,sofa_24hours
0,30004144,2126-04-04 13:00:00,2126-04-04 14:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
1,30004144,2126-04-04 14:00:00,2126-04-04 15:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
2,30004144,2126-04-04 15:00:00,2126-04-04 16:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
3,30004144,2126-04-04 16:00:00,2126-04-04 17:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
4,30004144,2126-04-04 17:00:00,2126-04-04 18:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
6043897,39993298,2189-08-03 11:00:00,2189-08-03 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0
6043898,39993298,2189-08-03 12:00:00,2189-08-03 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0
6043899,39993298,2189-08-03 13:00:00,2189-08-03 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0
6043900,39993298,2189-08-03 14:00:00,2189-08-03 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0


### mv

In [35]:
# pd.read_csv("/N/project/waveform_mortality/shared/datasets/Ventilation Status_MIMIC IV/mimic_iv_hourly_mv.csv")

In [79]:
mv = pd.read_csv("/N/project/waveform_mortality/YHS/data/new5_MIMICIV_hourly_MV.csv")
mv["starttime"] = pd.to_datetime(mv["hourly_timestamp"], errors="coerce")
mv["endtime"] = mv["starttime"] + pd.Timedelta(1, unit="h")
mv

,subject_id,stay_id,icu_intime,icu_outtime,status_starttime,status_endtime,hourly_timestamp,hour_rownum,ventilation_status,ventilation_category,MV_new_5_categories,starttime,endtime
0,12466550,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,2174-09-29 12:01:00,2174-09-29 20:00:00,2174-09-29 12:09:00,1,InvasiveVent,Invasive,invasive,2174-09-29 12:09:00,2174-09-29 13:09:00
1,12466550,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,2174-09-29 12:01:00,2174-09-29 20:00:00,2174-09-29 13:09:00,2,InvasiveVent,Invasive,invasive,2174-09-29 13:09:00,2174-09-29 14:09:00
2,12466550,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,2174-09-29 12:01:00,2174-09-29 20:00:00,2174-09-29 14:09:00,3,InvasiveVent,Invasive,invasive,2174-09-29 14:09:00,2174-09-29 15:09:00
3,12466550,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,2174-09-29 12:01:00,2174-09-29 20:00:00,2174-09-29 15:09:00,4,InvasiveVent,Invasive,invasive,2174-09-29 15:09:00,2174-09-29 16:09:00
4,12466550,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,2174-09-29 12:01:00,2174-09-29 20:00:00,2174-09-29 16:09:00,5,InvasiveVent,Invasive,invasive,2174-09-29 16:09:00,2174-09-29 17:09:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418673,19407684,37113957,2119-02-15 15:27:00,2119-03-02 17:00:56,2119-02-15 15:00:00,2119-02-28 20:00:00,2119-02-27 07:27:00,281,InvasiveVent,Invasive,invasive,2119-02-27 07:27:00,2119-02-27 08:27:00
5418674,19407684,37113957,2119-02-15 15:27:00,2119-03-02 17:00:56,2119-02-15 15:00:00,2119-02-28 20:00:00,2119-02-27 08:27:00,282,InvasiveVent,Invasive,invasive,2119-02-27 08:27:00,2119-02-27 09:27:00
5418675,19407684,37113957,2119-02-15 15:27:00,2119-03-02 17:00:56,2119-02-15 15:00:00,2119-02-28 20:00:00,2119-02-27 09:27:00,283,InvasiveVent,Invasive,invasive,2119-02-27 09:27:00,2119-02-27 10:27:00
5418676,19407684,37113957,2119-02-15 15:27:00,2119-03-02 17:00:56,2119-02-15 15:00:00,2119-02-28 20:00:00,2119-02-27 10:27:00,284,InvasiveVent,Invasive,invasive,2119-02-27 10:27:00,2119-02-27 11:27:00


In [80]:
mv["MV_new_5_categories"].unique()

array(['invasive', 'oxygen_therapy', 'none', 'non_invasive'], dtype=object)

In [82]:
mv = mv[["subject_id", "stay_id", "starttime", "endtime", "MV_new_5_categories"]]
mv

,subject_id,stay_id,starttime,endtime,MV_new_5_categories
0,12466550,30000153,2174-09-29 12:09:00,2174-09-29 13:09:00,invasive
1,12466550,30000153,2174-09-29 13:09:00,2174-09-29 14:09:00,invasive
2,12466550,30000153,2174-09-29 14:09:00,2174-09-29 15:09:00,invasive
3,12466550,30000153,2174-09-29 15:09:00,2174-09-29 16:09:00,invasive
4,12466550,30000153,2174-09-29 16:09:00,2174-09-29 17:09:00,invasive
...,...,...,...,...,...
5418673,19407684,37113957,2119-02-27 07:27:00,2119-02-27 08:27:00,invasive
5418674,19407684,37113957,2119-02-27 08:27:00,2119-02-27 09:27:00,invasive
5418675,19407684,37113957,2119-02-27 09:27:00,2119-02-27 10:27:00,invasive
5418676,19407684,37113957,2119-02-27 10:27:00,2119-02-27 11:27:00,invasive


### admission

In [83]:
admissions = pd.read_csv(os.path.join(data_dir, "hosp/admissions.csv"))
admissions["admittime"] = pd.to_datetime(admissions["admittime"], errors="coerce")
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"], errors="coerce")
admissions["deathtime"] = pd.to_datetime(admissions["deathtime"], errors="coerce")
admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [84]:
admissions["hospital_expire_flag"].unique()

array([0, 1])

In [85]:
admissions = admissions[["subject_id", "hadm_id", "deathtime", "race", "hospital_expire_flag"]]
admissions

,subject_id,hadm_id,deathtime,race,hospital_expire_flag
0,10000032,22595853,NaT,WHITE,0
1,10000032,22841357,NaT,WHITE,0
2,10000032,25742920,NaT,WHITE,0
3,10000032,29079034,NaT,WHITE,0
4,10000068,25022803,NaT,WHITE,0
...,...,...,...,...,...
431226,19999828,25744818,NaT,WHITE,0
431227,19999828,29734428,NaT,WHITE,0
431228,19999840,21033226,2164-09-17 13:42:00,WHITE,1
431229,19999840,26071774,NaT,WHITE,0


In [86]:
admissions["race"].value_counts()

WHITE                                        272932
BLACK/AFRICAN AMERICAN                        59959
OTHER                                         15102
UNKNOWN                                       10668
HISPANIC/LATINO - PUERTO RICAN                 8076
WHITE - OTHER EUROPEAN                         7925
HISPANIC OR LATINO                             7754
ASIAN                                          6156
ASIAN - CHINESE                                5587
WHITE - RUSSIAN                                5024
BLACK/CAPE VERDEAN                             4765
HISPANIC/LATINO - DOMINICAN                    4383
BLACK/CARIBBEAN ISLAND                         2704
BLACK/AFRICAN                                  2530
PATIENT DECLINED TO ANSWER                     1761
UNABLE TO OBTAIN                               1603
PORTUGUESE                                     1510
ASIAN - SOUTH EAST ASIAN                       1446
HISPANIC/LATINO - GUATEMALAN                   1330
ASIAN - ASIA

In [87]:
admissions.loc[admissions["race"].str.contains("WHITE"), "race"] = "WHITE"
admissions.loc[admissions["race"].str.contains("PORTUGUESE"), "race"] = "WHITE"

admissions.loc[admissions["race"].str.contains("BLACK"), "race"] = "BLACK"

admissions.loc[admissions["race"].str.contains("LATINO"), "race"] = "LATINO"
admissions.loc[admissions["race"].str.contains("SOUTH AMERICAN"), "race"] = "LATINO"

admissions.loc[admissions["race"].str.contains("ASIAN"), "race"] = "OTHER"

admissions.loc[admissions["race"].str.contains("UNKNOWN"), "race"] = "OTHER"
admissions.loc[admissions["race"].str.contains("INDIAN"), "race"] = "OTHER"
admissions.loc[admissions["race"].str.contains("HAWAIIAN"), "race"] = "OTHER"
admissions.loc[admissions["race"].str.contains("MULTIPLE RACE"), "race"] = "OTHER"
admissions.loc[admissions["race"].str.contains("UNABLE TO OBTAIN"), "race"] = "OTHER"
admissions.loc[admissions["race"].str.contains("PATIENT DECLINED TO ANSWER"), "race"] = "OTHER"
admissions["race"].value_counts()

WHITE     289664
BLACK      69958
OTHER      45892
LATINO     25717
Name: race, dtype: int64

In [43]:
# check race percent
# icustays_admissions_race = icustays.merge(admissions, on=["subject_id","hadm_id"], how="left")
# icustays.merge(admissions[["subject_id","hadm_id", "race"]], on=["subject_id","hadm_id"], how="left")
# icustays_admissions_race

,subject_id,hadm_id,stay_id,intime,outtime,los,gender,age,mi,chf,...,rend,canc,msld,metacanc,aids,CCI,height,deathtime,race,hospital_expire_flag
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,152.0,NaT,WHITE,0
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,5.0,NaN,NaT,BLACK,0
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaT,WHITE,0
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaT,WHITE,0
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,0,46,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,157.0,NaT,WHITE,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,1,43,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,193.0,NaT,WHITE,0
73177,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,1,82,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,3.0,NaN,NaT,WHITE,0
73178,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaT,WHITE,0
73179,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766,1,58,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,178.0,2164-09-17 13:42:00,WHITE,1


In [88]:
# icustays_admissions_race.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'los',
       'gender', 'age', 'mi', 'chf', 'pvd', 'cevd', 'dementia', 'cpd',
       'rheumd', 'pud', 'mld', 'diab', 'diabwc', 'hp', 'rend', 'canc', 'msld',
       'metacanc', 'aids', 'CCI', 'height', 'deathtime', 'race',
       'hospital_expire_flag'],
      dtype='object')

In [89]:
# len(icustays_admissions_race["subject_id"].unique())

50920

In [90]:
weights = weight_duration.groupby("stay_id")["weight"].max()
weights

stay_id
30000153     73.0
30000213     84.7
30000484     68.5
30000646     79.0
30001148     69.9
            ...  
39999301    107.7
39999384     67.0
39999552     69.8
39999562     62.0
39999810     72.0
Name: weight, Length: 70689, dtype: float64

In [47]:
weights.describe()

count    7.068900e+04
mean     9.924938e+01
std      4.148524e+03
min      1.000000e+00
25%      6.710000e+01
50%      8.000000e+01
75%      9.590000e+01
max      1.103030e+06
Name: weight, dtype: float64

In [48]:
sofa_scores

,stay_id,starttime,endtime,respiration,coagulation,liver,cardiovascular,cns,renal,sofa_24hours
0,30004144,2126-04-04 13:00:00,2126-04-04 14:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
1,30004144,2126-04-04 14:00:00,2126-04-04 15:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
2,30004144,2126-04-04 15:00:00,2126-04-04 16:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
3,30004144,2126-04-04 16:00:00,2126-04-04 17:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
4,30004144,2126-04-04 17:00:00,2126-04-04 18:00:00,0.0,0.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
6043897,39993298,2189-08-03 11:00:00,2189-08-03 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0
6043898,39993298,2189-08-03 12:00:00,2189-08-03 13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0
6043899,39993298,2189-08-03 13:00:00,2189-08-03 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0
6043900,39993298,2189-08-03 14:00:00,2189-08-03 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0


In [86]:
# worst_sofa_socre = []
# for n, g in sofa_scores.groupby("stay_id"):
#     data_in_g = g[g["hr"]<=24]
#     worst_sofa_socre.append({"stay_id": n, "worst_sofa_score_24h": data_in_g["sofa_24hours"].max()})

In [90]:
# worst_sofa_socre = pd.DataFrame(worst_sofa_socre)
# worst_sofa_socre
# worst_sofa_socre.describe()

,stay_id,worst_sofa_score_24h
count,7.316300e+04,73163.000000
mean,3.499293e+07,4.188251
std,2.889279e+06,3.228862
min,3.000015e+07,0.000000
25%,3.248971e+07,2.000000
50%,3.499439e+07,4.000000
75%,3.748847e+07,6.000000
max,3.999981e+07,22.000000


In [81]:
# icu_to_save.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'los',
       'gender', 'age', 'mi', 'chf', 'pvd', 'cevd', 'dementia', 'cpd',
       'rheumd', 'pud', 'mld', 'diab', 'diabwc', 'hp', 'rend', 'canc', 'msld',
       'metacanc', 'aids', 'CCI', 'height', 'deathtime', 'race',
       'hospital_expire_flag', 'weight', 'BMI'],
      dtype='object')

In [31]:
# icustays_admissions_race.drop_duplicates(["subject_id", "race"])["race"].value_counts()

WHITE     34635
OTHER      8748
BLACK      4681
LATINO     1810
ASIAN      1509
Name: race, dtype: int64

In [91]:
admissions["race_w"] = 0
admissions["race_b"] = 0
admissions["race_l"] = 0
admissions["race_o"] = 0
admissions.loc[admissions["race"]=="WHITE", "race_w"] = 1
admissions.loc[admissions["race"]=="BLACK", "race_b"] = 1
admissions.loc[admissions["race"]=="LATINO", "race_l"] = 1
admissions.loc[admissions["race"]=="OTHER", "race_o"] = 1
admissions

,subject_id,hadm_id,deathtime,race,hospital_expire_flag,race_w,race_b,race_l,race_o
0,10000032,22595853,NaT,WHITE,0,1,0,0,0
1,10000032,22841357,NaT,WHITE,0,1,0,0,0
2,10000032,25742920,NaT,WHITE,0,1,0,0,0
3,10000032,29079034,NaT,WHITE,0,1,0,0,0
4,10000068,25022803,NaT,WHITE,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,NaT,WHITE,0,1,0,0,0
431227,19999828,29734428,NaT,WHITE,0,1,0,0,0
431228,19999840,21033226,2164-09-17 13:42:00,WHITE,1,1,0,0,0
431229,19999840,26071774,NaT,WHITE,0,1,0,0,0


In [92]:
admissions = admissions.drop(columns=["race"], axis=1)
admissions

,subject_id,hadm_id,deathtime,hospital_expire_flag,race_w,race_b,race_l,race_o
0,10000032,22595853,NaT,0,1,0,0,0
1,10000032,22841357,NaT,0,1,0,0,0
2,10000032,25742920,NaT,0,1,0,0,0
3,10000032,29079034,NaT,0,1,0,0,0
4,10000068,25022803,NaT,0,1,0,0,0
...,...,...,...,...,...,...,...,...
431226,19999828,25744818,NaT,0,1,0,0,0
431227,19999828,29734428,NaT,0,1,0,0,0
431228,19999840,21033226,2164-09-17 13:42:00,1,1,0,0,0
431229,19999840,26071774,NaT,0,1,0,0,0


### chart events

In [93]:
chartevents = pd.read_csv(os.path.join(data_dir, "icu/chartevents.csv"))
chartevents["charttime"] = pd.to_datetime(chartevents["charttime"], errors="coerce")
chartevents

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82,82.000,mmHg,0.0
1,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220180,59,59.000,mmHg,0.0
2,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220181,63,63.000,mmHg,0.0
3,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220045,94,94.000,bpm,0.0
4,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85,85.000,mmHg,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313645058,19999987,23865745,36195440,NaN,2145-11-04 10:40:00,2145-11-04 11:28:00,225642,7.3,7.300,%,0.0
313645059,19999987,23865745,36195440,NaN,2145-11-04 10:40:00,2145-11-04 11:28:00,225643,68.6,68.600,%,0.0
313645060,19999987,23865745,36195440,NaN,2145-11-04 10:40:00,2145-11-04 11:28:00,227457,120,120.000,K/uL,1.0
313645061,19999987,23865745,36195440,NaN,2145-11-04 10:40:00,2145-11-04 11:30:00,220734,5.5,5.500,units,0.0


In [94]:
len(chartevents["subject_id"].unique()), len(chartevents["hadm_id"].unique()), len(chartevents["stay_id"].unique())

(50920, 66239, 73176)

In [95]:
def get_events(chartevents, item_ids):
    events = chartevents[chartevents["itemid"].isin(item_ids)]
    return events

def merge_events_with_weight_duration(events, weight_duration):
    events = events.merge(weight_duration, on="stay_id", how="left")
    events["valid_weight_flag"] = (events["starttime"] <= events["charttime"]) & (events["endtime"] >= events["charttime"])
    events = events[events["valid_weight_flag"]]
    return events.drop(columns=["starttime", "endtime", "valid_weight_flag"])

#### HR events

In [96]:
hr_item_ids = [220045]
hr_events = get_events(chartevents, hr_item_ids)
hr_events

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
3,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220045,94,94.0,bpm,0.0
9,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220045,97,97.0,bpm,0.0
19,10000032,29079034,39553978,66056.0,2180-07-23 20:00:00,2180-07-23 21:01:00,220045,100,100.0,bpm,0.0
60,10000032,29079034,39553978,66056.0,2180-07-23 21:00:00,2180-07-23 21:01:00,220045,94,94.0,bpm,0.0
166,10000032,29079034,39553978,88981.0,2180-07-23 14:12:00,2180-07-23 14:17:00,220045,91,91.0,bpm,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313644683,19999987,23865745,36195440,86756.0,2145-11-03 06:00:00,2145-11-03 06:05:00,220045,90,90.0,bpm,0.0
313644806,19999987,23865745,36195440,90295.0,2145-11-02 23:41:00,2145-11-02 23:41:00,220045,96,96.0,bpm,0.0
313644854,19999987,23865745,36195440,91879.0,2145-11-04 19:00:00,2145-11-04 19:30:00,220045,93,93.0,bpm,0.0
313644877,19999987,23865745,36195440,91879.0,2145-11-04 20:00:00,2145-11-04 20:02:00,220045,92,92.0,bpm,0.0


In [97]:
hr_events = hr_events[hr_events["valuenum"]<=300]

In [98]:
hr_events = hr_events[["subject_id", "hadm_id", "stay_id", "charttime", "valuenum"]]

In [99]:
hr_events = hr_events.merge(icustays, on=["subject_id", "hadm_id", "stay_id"], how="left")
hr_events

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,diab,diabwc,hp,rend,canc,msld,metacanc,aids,CCI,height
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6460811,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN
6460812,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN
6460813,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN
6460814,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN


In [100]:
len(hr_events["stay_id"].unique())

73163

In [101]:
hr_events = merge_events_with_weight_duration(hr_events, weight_duration[["stay_id", "weight", "starttime", "endtime"]])
hr_events

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,diabwc,hp,rend,canc,msld,metacanc,aids,CCI,height,weight
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96853673,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,88.6
96853675,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,94.0
96853680,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,94.2
96853683,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,94.2


In [46]:
mask = hr_events["height"].isna() | hr_events["weight"].isna()
mask.sum()

2278995

In [102]:
mask = hr_events["height"].notna() & hr_events["weight"].notna()
hr_events["bmi"] = np.nan
hr_events.loc[mask, "bmi"] =  hr_events.loc[mask, "weight"] / (hr_events.loc[mask, "height"]/100)**2
hr_events

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,hp,rend,canc,msld,metacanc,aids,CCI,height,weight,bmi
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4,17.053324
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4,17.053324
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4,17.053324
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4,17.053324
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,39.4,17.053324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96853673,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,88.6,NaN
96853675,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,94.0,NaN
96853680,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,94.2,NaN
96853683,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,94.2,NaN


In [103]:
len(hr_events.loc[hr_events["bmi"].isna(), "stay_id"].unique())

37384

In [104]:
hr_events.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'valuenum', 'intime',
       'outtime', 'los', 'gender', 'age', 'mi', 'chf', 'pvd', 'cevd',
       'dementia', 'cpd', 'rheumd', 'pud', 'mld', 'diab', 'diabwc', 'hp',
       'rend', 'canc', 'msld', 'metacanc', 'aids', 'CCI', 'height', 'weight',
       'bmi'],
      dtype='object')

In [105]:
hr_events.iloc[:,8:].isna().sum()

gender            0
age               0
mi             1870
chf            1870
pvd            1870
cevd           1870
dementia       1870
cpd            1870
rheumd         1870
pud            1870
mld            1870
diab           1870
diabwc         1870
hp             1870
rend           1870
canc           1870
msld           1870
metacanc       1870
aids           1870
CCI            1870
height      2278995
weight            0
bmi         2278995
dtype: int64

In [106]:
hr_events.columns[10:28]

Index(['mi', 'chf', 'pvd', 'cevd', 'dementia', 'cpd', 'rheumd', 'pud', 'mld',
       'diab', 'diabwc', 'hp', 'rend', 'canc', 'msld', 'metacanc', 'aids',
       'CCI'],
      dtype='object')

In [107]:
hr_events.iloc[:, 10:28] = hr_events.iloc[:, 10:28].fillna(0)

In [108]:
hr_events.columns[28:31]

Index(['height', 'weight', 'bmi'], dtype='object')

In [109]:
# availability columns
for c in hr_events.columns[28:31]:
    c_avail = f"{c}_avail"
    print(c)
    mask = hr_events[c].isna()
    hr_events.loc[mask, c_avail] = 0.0
    hr_events.loc[~mask, c_avail] = 1.0

hr_events

height
weight
bmi


,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,msld,metacanc,aids,CCI,height,weight,bmi,height_avail,weight_avail,bmi_avail
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96853673,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,3.0,NaN,88.6,NaN,0.0,1.0,0.0
96853675,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,3.0,NaN,94.0,NaN,0.0,1.0,0.0
96853680,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,3.0,NaN,94.2,NaN,0.0,1.0,0.0
96853683,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,3.0,NaN,94.2,NaN,0.0,1.0,0.0


In [110]:
# fill median
for c in hr_events.columns[28:31]:
    print(c)
    mask = hr_events[c].isna()
    
    median_to_fill = hr_events.loc[~mask, c].median()
    hr_events.loc[mask, c] = median_to_fill

height
weight
bmi


In [111]:
hr_events.iloc[:, 8:34]

,gender,age,mi,chf,pvd,cevd,dementia,cpd,rheumd,pud,...,msld,metacanc,aids,CCI,height,weight,bmi,height_avail,weight_avail,bmi_avail
0,0,52,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
1,0,52,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
2,0,52,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
3,0,52,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
4,0,52,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,152.0,39.4,17.053324,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96853673,0,57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,170.0,88.6,28.477509,0.0,1.0,0.0
96853675,0,57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,170.0,94.0,28.477509,0.0,1.0,0.0
96853680,0,57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,170.0,94.2,28.477509,0.0,1.0,0.0
96853683,0,57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,170.0,94.2,28.477509,0.0,1.0,0.0


In [112]:
for c in hr_events.columns:
    print(c)

subject_id
hadm_id
stay_id
charttime
valuenum
intime
outtime
los
gender
age
mi
chf
pvd
cevd
dementia
cpd
rheumd
pud
mld
diab
diabwc
hp
rend
canc
msld
metacanc
aids
CCI
height
weight
bmi
height_avail
weight_avail
bmi_avail


In [151]:
conn = sqlite3.connect(":memory:") 

In [152]:
# mv["ventilation_category"] = mv["MV_new_5_categories"]
mv["ventilation_category"].unique()

array(['invasive', 'oxygen_therapy', 'none', 'non_invasive'], dtype=object)

In [153]:
mv.to_sql("mv", conn, index=False)
hr_events.to_sql("hr", conn, index=False)
sofa_scores.to_sql("sofa", conn, index=False)

6043902

In [156]:
query = """
    SELECT hr.*, mv.ventilation_category, 
    sofa.respiration, sofa.coagulation, sofa.liver, sofa.cardiovascular, sofa.cns, sofa.renal, sofa.sofa_24hours
    FROM hr
    LEFT JOIN mv
    ON hr.stay_id=mv.stay_id and hr.charttime>=mv.starttime and hr.charttime<mv.endtime
    LEFT JOIN sofa
    ON hr.stay_id=sofa.stay_id and hr.charttime>=sofa.starttime and hr.charttime<sofa.endtime
"""

hr_events_mv_sofa = pd.read_sql_query(query, conn)
hr_events_mv_sofa

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,weight_avail,bmi_avail,ventilation_category,respiration,coagulation,liver,cardiovascular,cns,renal,sofa_24hours
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,1.0,none,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,1.0,oxygen_therapy,0.0,0.0,0.0,1.0,0.0,0.0,2.0
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,1.0,oxygen_therapy,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,1.0,none,0.0,0.0,0.0,1.0,0.0,0.0,2.0
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,1.0,none,0.0,0.0,0.0,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496474,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,1.0,0.0,invasive,0.0,0.0,0.0,0.0,0.0,0.0,6.0
6496475,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,1.0,0.0,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6496476,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,1.0,0.0,oxygen_therapy,0.0,0.0,0.0,0.0,1.0,0.0,7.0
6496477,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,1.0,0.0,oxygen_therapy,0.0,0.0,0.0,0.0,0.0,0.0,7.0


In [150]:
conn.close()

In [157]:
len(hr_events["stay_id"].unique()), len(hr_events_mv_sofa["stay_id"].unique())

(70685, 70685)

In [158]:
hr_events.groupby(by=["stay_id", "charttime"]).ngroups, hr_events_mv_sofa.groupby(by=["stay_id", "charttime"]).ngroups

(6351787, 6351787)

In [159]:
ventilation_category = hr_events_mv_sofa["ventilation_category"]
ventilation_category

0                    none
1          oxygen_therapy
2          oxygen_therapy
3                    none
4                    none
                ...      
6496474          invasive
6496475              none
6496476    oxygen_therapy
6496477    oxygen_therapy
6496478              none
Name: ventilation_category, Length: 6496479, dtype: object

In [160]:
ventilation_category.unique()

array(['none', 'oxygen_therapy', None, 'invasive', 'non_invasive'],
      dtype=object)

In [161]:
ventilation_category.value_counts()

invasive          2200701
none              1838410
oxygen_therapy    1736045
non_invasive        39639
Name: ventilation_category, dtype: int64

In [162]:
hr_events_mv_sofa[ventilation_category.isna()]

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,weight_avail,bmi_avail,ventilation_category,respiration,coagulation,liver,cardiovascular,cns,renal,sofa_24hours
10,10000980,26913865,39765666,2189-06-27 08:56:00,77.0,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,10000980,26913865,39765666,2189-06-27 09:04:00,75.0,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,10000980,26913865,39765666,2189-06-27 10:00:00,73.0,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,10000980,26913865,39765666,2189-06-27 11:00:00,74.0,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,10000980,26913865,39765666,2189-06-27 12:00:00,75.0,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,0,76,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496258,19999828,25744818,36075953,2149-01-10 03:00:00,77.0,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6496259,19999828,25744818,36075953,2149-01-10 04:00:00,82.0,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6496260,19999828,25744818,36075953,2149-01-10 05:00:00,80.0,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6496261,19999828,25744818,36075953,2149-01-10 06:00:00,79.0,2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995,0,48,...,1.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [165]:
hr_events_mv_sofa["mv_invasive"] = 0
hr_events_mv_sofa["mv_non_vasive"] = 0
hr_events_mv_sofa["mv_oxygen_therapy"] = 0
hr_events_mv_sofa["mv_none"] = 0
hr_events_mv_sofa["mv_unknown"] = 0
hr_events_mv_sofa.loc[ventilation_category == "invasive", "mv_invasive"] = 1
hr_events_mv_sofa.loc[ventilation_category == "non_invasive", "mv_non_vasive"] = 1
hr_events_mv_sofa.loc[ventilation_category == "oxygen_therapy", "mv_oxygen_therapy"] = 1
hr_events_mv_sofa.loc[ventilation_category == "none", "mv_none"] = 1
hr_events_mv_sofa.loc[ventilation_category.isna(), "mv_unknown"] = 1
hr_events_mv_sofa

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,liver,cardiovascular,cns,renal,sofa_24hours,mv_invasive,mv_non_vasive,mv_none,mv_unknown,mv_oxygen_therapy
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,2.0,0,0,1,0,0
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,1.0,0.0,0.0,2.0,0,0,0,0,1
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,2.0,0,0,0,0,1
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,1.0,0.0,0.0,2.0,0,0,1,0,0
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,1.0,1.0,0.0,2.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496474,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,6.0,1,0,0,0,0
6496475,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0
6496476,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,1.0,0.0,7.0,0,0,0,0,1
6496477,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,7.0,0,0,0,0,1


In [166]:
hr_events_mv_sofa.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'valuenum', 'intime',
       'outtime', 'los', 'gender', 'age', 'mi', 'chf', 'pvd', 'cevd',
       'dementia', 'cpd', 'rheumd', 'pud', 'mld', 'diab', 'diabwc', 'hp',
       'rend', 'canc', 'msld', 'metacanc', 'aids', 'CCI', 'height', 'weight',
       'bmi', 'height_avail', 'weight_avail', 'bmi_avail',
       'ventilation_category', 'respiration', 'coagulation', 'liver',
       'cardiovascular', 'cns', 'renal', 'sofa_24hours', 'mv_invasive',
       'mv_non_vasive', 'mv_none', 'mv_unknown', 'mv_oxygen_therapy'],
      dtype='object')

In [167]:
hr_events_mv_sofa = hr_events_mv_sofa.drop(columns=["ventilation_category"])
hr_events_mv_sofa

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,liver,cardiovascular,cns,renal,sofa_24hours,mv_invasive,mv_non_vasive,mv_none,mv_unknown,mv_oxygen_therapy
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,2.0,0,0,1,0,0
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,1.0,0.0,0.0,2.0,0,0,0,0,1
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,0.0,2.0,0,0,0,0,1
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,1.0,0.0,0.0,2.0,0,0,1,0,0
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,1.0,1.0,0.0,2.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496474,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,6.0,1,0,0,0,0
6496475,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0
6496476,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,1.0,0.0,7.0,0,0,0,0,1
6496477,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,7.0,0,0,0,0,1


In [168]:
hr_events_mv_sofa["charttime"] = pd.to_datetime(hr_events_mv_sofa["charttime"], errors="coerce")
hr_events_mv_sofa["intime"] = pd.to_datetime(hr_events_mv_sofa["intime"], errors="coerce")

In [169]:
hr_events_mv_sofa["time_diff"] = hr_events_mv_sofa["charttime"] - hr_events_mv_sofa["intime"]
hr_events_mv_sofa

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,cardiovascular,cns,renal,sofa_24hours,mv_invasive,mv_non_vasive,mv_none,mv_unknown,mv_oxygen_therapy,time_diff
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,2.0,0,0,1,0,0,0 days 08:00:00
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,0.0,0.0,2.0,0,0,0,0,1,0 days 05:00:00
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0.0,0.0,0.0,2.0,0,0,0,0,1,0 days 06:00:00
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,0.0,0.0,2.0,0,0,1,0,0,0 days 07:00:00
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1.0,1.0,0.0,2.0,0,0,1,0,0,0 days 00:12:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496474,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,6.0,1,0,0,0,0,0 days 07:01:00
6496475,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,0.0,0,0,1,0,0,0 days 00:42:00
6496476,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,1.0,0.0,7.0,0,0,0,0,1,1 days 20:01:00
6496477,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0.0,0.0,0.0,7.0,0,0,0,0,1,1 days 21:01:00


In [170]:
hr_events_mv_sofa.iloc[:, 34:41].isna().sum()

respiration       8025
coagulation       8025
liver             8025
cardiovascular    8025
cns               8025
renal             8025
sofa_24hours      8025
dtype: int64

In [171]:
hr_events_mv_sofa.iloc[:, 34:41] = hr_events_mv_sofa.iloc[:, 34:41].fillna(0) 

In [172]:
hr_events_mv_sofa.iloc[:, 34:41].isna().sum()

respiration       0
coagulation       0
liver             0
cardiovascular    0
cns               0
renal             0
sofa_24hours      0
dtype: int64

In [173]:
merged_hr_events = hr_events_mv_sofa.merge(admissions, on=["subject_id", "hadm_id"])
merged_hr_events

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,gender,age,...,mv_none,mv_unknown,mv_oxygen_therapy,time_diff,deathtime,hospital_expire_flag,race_w,race_b,race_l,race_o
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1,0,0,0 days 08:00:00,NaT,0,1,0,0,0
1,10000032,29079034,39553978,2180-07-23 19:00:00,97.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0,0,1,0 days 05:00:00,NaT,0,1,0,0,0
2,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,0,0,1,0 days 06:00:00,NaT,0,1,0,0,0
3,10000032,29079034,39553978,2180-07-23 21:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1,0,0,0 days 07:00:00,NaT,0,1,0,0,0
4,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,52,...,1,0,0,0 days 00:12:00,NaT,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496474,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0,0,0,0 days 07:01:00,NaT,0,0,0,0,1
6496475,19999987,23865745,36195440,2145-11-02 23:41:00,96.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,1,0,0,0 days 00:42:00,NaT,0,0,0,0,1
6496476,19999987,23865745,36195440,2145-11-04 19:00:00,93.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0,0,1,1 days 20:01:00,NaT,0,0,0,0,1
6496477,19999987,23865745,36195440,2145-11-04 20:00:00,92.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,57,...,0,0,1,1 days 21:01:00,NaT,0,0,0,0,1


In [174]:
merged_hr_events.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'valuenum', 'intime',
       'outtime', 'los', 'gender', 'age', 'mi', 'chf', 'pvd', 'cevd',
       'dementia', 'cpd', 'rheumd', 'pud', 'mld', 'diab', 'diabwc', 'hp',
       'rend', 'canc', 'msld', 'metacanc', 'aids', 'CCI', 'height', 'weight',
       'bmi', 'height_avail', 'weight_avail', 'bmi_avail', 'respiration',
       'coagulation', 'liver', 'cardiovascular', 'cns', 'renal',
       'sofa_24hours', 'mv_invasive', 'mv_non_vasive', 'mv_none', 'mv_unknown',
       'mv_oxygen_therapy', 'time_diff', 'deathtime', 'hospital_expire_flag',
       'race_w', 'race_b', 'race_l', 'race_o'],
      dtype='object')

In [175]:
merged_hr_events.iloc[:, 8:].isna().sum()

gender                        0
age                           0
mi                            0
chf                           0
pvd                           0
cevd                          0
dementia                      0
cpd                           0
rheumd                        0
pud                           0
mld                           0
diab                          0
diabwc                        0
hp                            0
rend                          0
canc                          0
msld                          0
metacanc                      0
aids                          0
CCI                           0
height                        0
weight                        0
bmi                           0
height_avail                  0
weight_avail                  0
bmi_avail                     0
respiration                   0
coagulation                   0
liver                         0
cardiovascular                0
cns                           0
renal   

In [182]:
for h in [6, 12, 24, 24*2, 24*3, 24*4, 24*5, 24*6, 24*7, 24*8, 24*9, 24*10, 7200]:
    merged_hr_events[f"death_in_{h}_hours"] = (merged_hr_events["charttime"] + pd.Timedelta(hours=h)) >= merged_hr_events["deathtime"]

In [183]:
for h in [6, 12, 24, 24*2, 24*3, 24*4, 24*5, 24*6, 24*7, 24*8, 24*9, 24*10, 7200]:
    merged_hr_events.loc[merged_hr_events["deathtime"].isna(), f"death_in_{h}_hours"] = False

In [184]:
for h in [6, 12, 24, 24*2, 24*3, 24*4, 24*5, 24*6, 24*7, 24*8, 24*9, 24*10, 7200]:
    merged_hr_events[f"death_in_{h}_hours"] = merged_hr_events[f"death_in_{h}_hours"].astype(int)

In [185]:
for h in [6, 12, 24, 24*2, 24*3, 24*4, 24*5, 24*6, 24*7, 24*8, 24*9, 24*10, 7200]:
    print(merged_hr_events[f"death_in_{h}_hours"].value_counts())
print(merged_hr_events["hospital_expire_flag"].value_counts())

0    6445355
1      51124
Name: death_in_6_hours, dtype: int64
0    6410781
1      85698
Name: death_in_12_hours, dtype: int64
0    6348196
1     148283
Name: death_in_24_hours, dtype: int64
0    6239956
1     256523
Name: death_in_48_hours, dtype: int64
0    6146660
1     349819
Name: death_in_72_hours, dtype: int64
0    6065750
1     430729
Name: death_in_96_hours, dtype: int64
0    5994748
1     501731
Name: death_in_120_hours, dtype: int64
0    5932884
1     563595
Name: death_in_144_hours, dtype: int64
0    5877628
1     618851
Name: death_in_168_hours, dtype: int64
0    5829280
1     667199
Name: death_in_192_hours, dtype: int64
0    5786431
1     710048
Name: death_in_216_hours, dtype: int64
0    5748285
1     748194
Name: death_in_240_hours, dtype: int64
0    5378516
1    1117963
Name: death_in_7200_hours, dtype: int64
0    5377707
1    1118772
Name: hospital_expire_flag, dtype: int64


In [186]:
for c in merged_hr_events.columns:
    print(f'"{c}",')

"subject_id",
"hadm_id",
"stay_id",
"charttime",
"valuenum",
"intime",
"outtime",
"los",
"gender",
"age",
"mi",
"chf",
"pvd",
"cevd",
"dementia",
"cpd",
"rheumd",
"pud",
"mld",
"diab",
"diabwc",
"hp",
"rend",
"canc",
"msld",
"metacanc",
"aids",
"CCI",
"height",
"weight",
"bmi",
"height_avail",
"weight_avail",
"bmi_avail",
"respiration",
"coagulation",
"liver",
"cardiovascular",
"cns",
"renal",
"sofa_24hours",
"mv_invasive",
"mv_non_vasive",
"mv_none",
"mv_unknown",
"mv_oxygen_therapy",
"time_diff",
"deathtime",
"hospital_expire_flag",
"race_w",
"race_b",
"race_l",
"race_o",
"death_in_6_hours",
"death_in_24_hours",
"death_in_48_hours",
"death_in_72_hours",
"death_in_96_hours",
"death_in_120_hours",
"death_in_144_hours",
"death_in_168_hours",
"death_in_192_hours",
"death_in_216_hours",
"death_in_240_hours",
"death_in_7200_hours",
"death_in_12_hours",


In [187]:
dst_dir = "/N/project/waveform_mortality/xiang/Projects/icu-contour/data"
merged_hr_events.to_csv(os.path.join(dst_dir, "merged_hr_events_mv_5.csv"))

### SBP

In [143]:
sbp_item_ids = [220179, 220050, 225309]
sbp_events = get_events(chartevents, sbp_item_ids)
sbp_events

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82,82.0,mmHg,0.0
4,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85,85.0,mmHg,0.0
10,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220179,93,93.0,mmHg,0.0
21,10000032,29079034,39553978,66056.0,2180-07-23 20:00:00,2180-07-23 21:01:00,220179,90,90.0,mmHg,0.0
163,10000032,29079034,39553978,88981.0,2180-07-23 14:11:00,2180-07-23 14:17:00,220179,84,84.0,mmHg,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313644546,19999987,23865745,36195440,86756.0,2145-11-03 04:00:00,2145-11-03 04:24:00,220179,106,106.0,mmHg,0.0
313644654,19999987,23865745,36195440,86756.0,2145-11-03 05:00:00,2145-11-03 05:05:00,220179,113,113.0,mmHg,0.0
313644725,19999987,23865745,36195440,86756.0,2145-11-03 06:00:00,2145-11-03 06:46:00,220179,110,110.0,mmHg,0.0
313644858,19999987,23865745,36195440,91879.0,2145-11-04 19:31:00,2145-11-04 19:38:00,220179,163,163.0,mmHg,0.0


In [144]:
# sbp 30 - 300 
# dbp 20 - 200
sbp_events = sbp_events[(sbp_events["valuenum"]>=30) & (sbp_events["valuenum"]<=300)]

In [145]:
sbp_events = sbp_events[["subject_id", "hadm_id", "stay_id", "charttime", "valuenum"]]

In [146]:
sbp_events = sbp_events.merge(icustays, on=["subject_id", "hadm_id", "stay_id"], how="left")
sbp_events

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,icu_history,gender,...,diab,diabwc,hp,rend,canc,msld,metacanc,aids,CCI,height
0,10000032,29079034,39553978,2180-07-23 21:01:00,82.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
1,10000032,29079034,39553978,2180-07-23 22:00:00,85.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
2,10000032,29079034,39553978,2180-07-23 19:00:00,93.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
3,10000032,29079034,39553978,2180-07-23 20:00:00,90.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
4,10000032,29079034,39553978,2180-07-23 14:11:00,84.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6558451,19999987,23865745,36195440,2145-11-03 04:00:00,106.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN
6558452,19999987,23865745,36195440,2145-11-03 05:00:00,113.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN
6558453,19999987,23865745,36195440,2145-11-03 06:00:00,110.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN
6558454,19999987,23865745,36195440,2145-11-04 19:31:00,163.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN


In [147]:
sbp_events["time_diff"] = sbp_events["charttime"] - sbp_events["intime"]
sbp_events

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,icu_history,gender,...,diabwc,hp,rend,canc,msld,metacanc,aids,CCI,height,time_diff
0,10000032,29079034,39553978,2180-07-23 21:01:00,82.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,0 days 07:01:00
1,10000032,29079034,39553978,2180-07-23 22:00:00,85.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,0 days 08:00:00
2,10000032,29079034,39553978,2180-07-23 19:00:00,93.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,0 days 05:00:00
3,10000032,29079034,39553978,2180-07-23 20:00:00,90.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,0 days 06:00:00
4,10000032,29079034,39553978,2180-07-23 14:11:00,84.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,152.0,0 days 00:11:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6558451,19999987,23865745,36195440,2145-11-03 04:00:00,106.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,0 days 05:01:00
6558452,19999987,23865745,36195440,2145-11-03 05:00:00,113.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,0 days 06:01:00
6558453,19999987,23865745,36195440,2145-11-03 06:00:00,110.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,0 days 07:01:00
6558454,19999987,23865745,36195440,2145-11-04 19:31:00,163.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,NaN,1 days 20:32:00


In [148]:
merged_sbp_events = sbp_events.merge(admissions, on=["subject_id", "hadm_id"])
merged_sbp_events

,subject_id,hadm_id,stay_id,charttime,valuenum,intime,outtime,los,icu_history,gender,...,aids,CCI,height,time_diff,deathtime,hospital_expire_flag,race_w,race_b,race_l,race_o
0,10000032,29079034,39553978,2180-07-23 21:01:00,82.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,2.0,152.0,0 days 07:01:00,NaT,0,1,0,0,0
1,10000032,29079034,39553978,2180-07-23 22:00:00,85.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,2.0,152.0,0 days 08:00:00,NaT,0,1,0,0,0
2,10000032,29079034,39553978,2180-07-23 19:00:00,93.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,2.0,152.0,0 days 05:00:00,NaT,0,1,0,0,0
3,10000032,29079034,39553978,2180-07-23 20:00:00,90.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,2.0,152.0,0 days 06:00:00,NaT,0,1,0,0,0
4,10000032,29079034,39553978,2180-07-23 14:11:00,84.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0.0,2.0,152.0,0 days 00:11:00,NaT,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6558451,19999987,23865745,36195440,2145-11-03 04:00:00,106.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,3.0,NaN,0 days 05:01:00,NaT,0,0,0,0,1
6558452,19999987,23865745,36195440,2145-11-03 05:00:00,113.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,3.0,NaN,0 days 06:01:00,NaT,0,0,0,0,1
6558453,19999987,23865745,36195440,2145-11-03 06:00:00,110.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,3.0,NaN,0 days 07:01:00,NaT,0,0,0,0,1
6558454,19999987,23865745,36195440,2145-11-04 19:31:00,163.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0.0,3.0,NaN,1 days 20:32:00,NaT,0,0,0,0,1


In [149]:
merged_sbp_events.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'valuenum', 'intime',
       'outtime', 'los', 'icu_history', 'gender', 'age', 'mi', 'chf', 'pvd',
       'cevd', 'dementia', 'cpd', 'rheumd', 'pud', 'mld', 'diab', 'diabwc',
       'hp', 'rend', 'canc', 'msld', 'metacanc', 'aids', 'CCI', 'height',
       'time_diff', 'deathtime', 'hospital_expire_flag', 'race_w', 'race_b',
       'race_l', 'race_o'],
      dtype='object')

In [196]:
dst_dir = "/N/project/waveform_mortality/xiang/Projects/icu-contour/data"
merged_sbp_events.to_csv(os.path.join(dst_dir, "merged_sbp_events.csv"))

## Dataset

### Check # subjects

In [151]:
hr_suject_ids = merged_hr_events["subject_id"].unique()
len(hr_suject_ids)

50872

In [152]:
sbp_subject_ids = merged_sbp_events["subject_id"].unique()
len(sbp_subject_ids)

50911

In [153]:
def merge_measurements(measurement_1, measurement_2, start_time, end_time):
    m1_clean = measurement_1[(measurement_1["time_diff"] >= start_time) & (measurement_1["time_diff"] < end_time)]
    m2_clean = measurement_2[(measurement_2["time_diff"] >= start_time) & (measurement_2["time_diff"] < end_time)]
    m1_clean.rename(columns={"valuenum": "m1_valuenum"}, inplace=True)
    m2_clean = m2_clean[["stay_id", "valuenum"]]
    m2_clean.rename(columns={"valuenum": "m2_valuenum"}, inplace=True)
    return pd.merge(m1_clean, m2_clean, on="stay_id")

In [183]:
prediction_window, mop, outcome_window = 6, 6, 6
start_time = pd.to_timedelta(mop, unit="h")
end_time = start_time + pd.to_timedelta(prediction_window, unit="h")

In [156]:
merged_measurements = merge_measurements(
    merged_hr_events, merged_sbp_events, start_time, end_time
)
merged_measurements

,subject_id,hadm_id,stay_id,charttime,m1_valuenum,intime,outtime,los,icu_history,gender,...,race_l,race_o,death_in_6_hours,death_in_12_hours,death_in_24_hours,death_in_36_hours,death_in_48_hours,death_in_60_hours,death_in_72_hours,m2_valuenum
0,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0,0,0,False,0,0,0,0,0,82.0
1,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0,0,0,False,0,0,0,0,0,85.0
2,10000032,29079034,39553978,2180-07-23 22:00:00,94.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0,0,0,False,0,0,0,0,0,90.0
3,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0,0,0,False,0,0,0,0,0,82.0
4,10000032,29079034,39553978,2180-07-23 20:00:00,100.0,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,0,0,...,0,0,0,False,0,0,0,0,0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092247,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0,1,0,False,0,0,0,0,0,106.0
4092248,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0,1,0,False,0,0,0,0,0,90.0
4092249,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0,1,0,False,0,0,0,0,0,110.0
4092250,19999987,23865745,36195440,2145-11-03 06:00:00,90.0,2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,0,0,...,0,1,0,False,0,0,0,0,0,113.0


In [158]:
# measurements median
measurements = merged_measurements.groupby("stay_id").agg({
    "m1_valuenum": "median", "m2_valuenum": "median"
})
measurements

,m1_valuenum,m2_valuenum
stay_id,,
30000153,123.0,136.0
30000213,84.0,127.0
30000484,84.5,91.0
30000646,78.0,90.0
30001148,80.0,108.0
...,...,...
39999301,58.0,116.0
39999384,69.0,158.0
39999552,90.0,128.0


In [166]:
mask = (measurements["m1_valuenum"]>=50) & (measurements["m1_valuenum"]<=60) & (measurements["m2_valuenum"]>=45) & (measurements["m2_valuenum"]<=50)
measurements[mask]

,m1_valuenum,m2_valuenum
stay_id,,
35679245,52.5,46.5
38939945,50.0,50.0


In [167]:
stay_ids = list(measurements[mask].index)
stay_ids

[35679245, 38939945]

In [179]:
merged_hr_events[merged_hr_events["stay_id"].isin(stay_ids)][["subject_id", "stay_id", "valuenum", "gender", "age", "race_w", "race_b", "race_l", "race_o"]]

,subject_id,stay_id,valuenum,gender,age,race_w,race_b,race_l,race_o
221644,10352831,38939945,50.0,1,85,0,1,0,0
221645,10352831,38939945,50.0,1,85,0,1,0,0
221646,10352831,38939945,50.0,1,85,0,1,0,0
221647,10352831,38939945,50.0,1,85,0,1,0,0
221648,10352831,38939945,49.0,1,85,0,1,0,0
221649,10352831,38939945,50.0,1,85,0,1,0,0
221650,10352831,38939945,51.0,1,85,0,1,0,0
221651,10352831,38939945,50.0,1,85,0,1,0,0
221652,10352831,38939945,50.0,1,85,0,1,0,0
221653,10352831,38939945,59.0,1,85,0,1,0,0


In [178]:
merged_hr_events[merged_hr_events["stay_id"].isin(stay_ids)].iloc[:, 11:29]

,mi,chf,pvd,cevd,dementia,cpd,rheumd,pud,mld,diab,diabwc,hp,rend,canc,msld,metacanc,aids,CCI
221644,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221645,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221646,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221647,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221648,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221649,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221650,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221651,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221652,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
221653,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0


In [176]:
merged_hr_events[merged_hr_events["stay_id"].isin(stay_ids)].iloc[:, 35:42]

,respiration,coagulation,liver,cardiovascular,cns,renal,sofa_24hours
221644,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221645,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221646,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221647,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221648,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221649,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221650,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221651,0.0,0.0,0.0,4.0,0.0,0.0,11.0
221652,0.0,0.0,0.0,1.0,0.0,0.0,11.0
221653,0.0,0.0,0.0,1.0,0.0,0.0,11.0


In [186]:
prediction_window, mop, outcome_window = 6, 6, 12
data_dir = f"/N/project/waveform_mortality/xiang/Projects/icu-contour/results_icu_history/death_in_{outcome_window}_hours_smote_smooth"
npz_data = np.load(os.path.join(data_dir, f"auc_fpr_tpr__PW_{prediction_window}__MOP_{mop}__OW_{outcome_window}.npz"))
auc, fpr, tpr, thresholds = npz_data["auc"], npz_data["fpr"], npz_data["tpr"], npz_data["thresholds"]
thresholds

array([1.98, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9 , 0.89,
       0.88, 0.87, 0.86, 0.85, 0.84, 0.83, 0.82, 0.81, 0.8 , 0.79, 0.78,
       0.77, 0.76, 0.75, 0.74, 0.73, 0.72, 0.71, 0.7 , 0.69, 0.68, 0.67,
       0.66, 0.65, 0.64, 0.63, 0.62, 0.61, 0.6 , 0.59, 0.58, 0.57, 0.56,
       0.55, 0.54, 0.53, 0.52, 0.51, 0.5 , 0.49, 0.48, 0.47, 0.46, 0.45,
       0.44, 0.43, 0.42, 0.41, 0.39, 0.38, 0.37, 0.36, 0.35, 0.34, 0.33,
       0.32, 0.31, 0.3 , 0.29, 0.28, 0.27, 0.26, 0.25, 0.24, 0.23, 0.22,
       0.21, 0.2 , 0.19, 0.18, 0.17, 0.16, 0.15, 0.14, 0.13, 0.12, 0.11,
       0.1 , 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01, 0.  ])